# Пример расчета времени инференса детектора

## Шаг 1: Установка библиотеки

In [ ]:
!pip install manuscript-ocr

## Шаг 2: Загрузка тестовых данных

In [ ]:
import requests, zipfile


# 2) Скачиваем архив с Яндекс.Диска и распаковываем
share_link = "https://disk.yandex.ru/d/tUFCt2X6iWPS_w"
api_url    = "https://cloud-api.yandex.net/v1/disk/public/resources/download"

# получаем прямую ссылку
download_url = requests.get(api_url, params={"public_key": share_link}).json()["href"]

# загружаем и сразу распаковываем в /content/data
resp = requests.get(download_url)
with open("/content/data.zip", "wb") as f:
    f.write(resp.content)

with zipfile.ZipFile("/content/data.zip", "r") as z:
    z.extractall("/content/data")

## Шаг 3: Инициализация детектора и замер времени

In [ ]:
import os
import time
import numpy as np
from manuscript.detectors import EAST

# Папка с распакованными изображениями
extract_dir = "/content/data/Архивы (формат для обучения детектора)/test_images/"

# Расширения изображений
IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif")

# Инициализация детектора
detector = EAST()

# Инференс по всем изображениям и замер времени
times = []
for root, _, files in os.walk(extract_dir):
    for fname in files:
        if fname.lower().endswith(IMAGE_EXTS):
            img_path = os.path.join(root, fname)
            print(img_path)
            start = time.time()
            result = detector.predict(img_path)
            times.append(time.time() - start)

# Статистика времени
times = np.array(times)
print(f"Обработано изображений: {len(times)}")
print(f"Мин.: {times.min():.3f} сек, Макс.: {times.max():.3f} сек, Ср.: {times.mean():.3f} сек")


Loaded pretrained model from /root/.east_weights.pth
/content/data/Архивы (формат для обучения детектора)/test_images/1320.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/12596.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/6478.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/4984.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/481.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/12319.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/5238.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/1369.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/12344.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/1054.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/419.jpg
/content/data/Архивы (формат для обучения детектора)/test_images/8920.jpg
/content/data/Архивы (формат для обучения детектора)/test_

## Шаг 4: Инференс с визуализацией результата

Теперь выполним полный OCR (детекция + распознавание) одного изображения и визуализируем результат.

In [ ]:
from manuscript import Pipeline
from manuscript.utils.visualization import visualize_page
from manuscript.detectors import EAST
from manuscript.recognizers import TRBA

# Выбираем случайное изображение из загруженных
image_path = img_path  # используем последний путь из предыдущего цикла
# или можно указать конкретный путь, например:
# image_path = "/content/data/Архивы (формат для обучения детектора)/test_images/403.jpg"

# Создание OCR-пайплайна с моделями по умолчанию
pipeline = Pipeline(
    detector=EAST(),
    recognizer=TRBA(weights="trba_lite_g1"),
)

# Обработка изображения и получение результата
result = pipeline.predict(image_path)

print("Результат:")
print(result)
print("\n" + "="*50 + "\n")

# Извлечение распознанного текста
text = pipeline.get_text(result["page"])
print("Распознанный текст:")
print(text)
print("\n" + "="*50 + "\n")

# Визуализация результата
vis_img = visualize_page(image_path, result["page"])
vis_img.show()  # или vis_img.save("result.jpg") для сохранения